# Cookbook: Database Manipulation

In this cookbook, we will explore a simple data processing example: reading a file, adding its data to a database, and querying it. It shows how we can efficiently manipulate data with agents.

# Install Requirements

We will use [Qwen-Agent](https://github.com/QwenLM/Qwen-Agent) in this book. For demonstration, we will need at least the `mcp` functionality of Qwen-Agent.

In [ ]:
!pip3 install -U "qwen-agent[gui,rag,code_interpreter,mcp]"
# `pip install -U qwen-agent` will install the minimal requirements.
# The optional requirements, specified in double brackets, are:
#   [gui] for Gradio-based GUI support;
#   [rag] for RAG support;
#   [code_interpreter] for Code Interpreter support;
#   [mcp] for MCP support.

In [1]:
!pip3 install -U uv
# We will use mcp servers that run with the `uvx` command, so `uv` is required.

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 15.6 MB 643 kB/s eta 0:00:01
You should consider upgrading via the '/Users/xieming/Desktop/qwen-agent/Qwen-Agent/venv/bin/python3 -m pip install --upgrade pip' command.


In [4]:
!npm --version
# We also use mcp servers that run with the `npx` command, so node.js should be installed.

10.9.2


# Create a Spreadsheet

We create a sample spreadsheet and save it as `scores.csv` for later use.

In [5]:
import csv

csv_file_name = 'scores.csv'
table = [
    {"id": 100, "score": 85},
    {"id": 101, "score": 90},
    {"id": 102, "score": 88},
    {"id": 103, "score": 95},
    {"id": 104, "score": 80},
    {"id": 105, "score": 92},
    {"id": 106, "score": 87},
    {"id": 107, "score": 100},
    {"id": 108, "score": 83},
    {"id": 109, "score": 98}
]

with open(csv_file_name, mode='w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=['id', 'score'])
    writer.writeheader()
    for record in table:
        writer.writerow(record)

# Step 1: Create an Agent

Create an agent that is capable of 
- listing, reading and writing files (through the MCP server `@modelcontextprotocol/server-filesystem`).
- performing SQLite database manipulations (through the MCP server `mcp-server-sqlite`).

In [6]:
from qwen_agent.agents import Assistant
from qwen_agent.utils.output_beautify import typewriter_print
# `typewriter_print` prints streaming messages in a non-overlapping manner for a clear view.

/Users/xieming/Desktop/qwen-agent/Qwen-Agent/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
llm_cfg = {
    'model': 'qwen3-32b',
    'model_server': 'dashscope',
    'api_key': '' # **fill your dashscope api key here**

    # Use a model service compatible with the OpenAI API, such as vLLM or Ollama:
    # 'model': 'Qwen3-8B',
    # 'model_server': 'http://localhost:8000/v1',  # base_url, also known as api_base
    # 'api_key': 'EMPTY'
}
llm_cfg={'model':'qwen3:8b', 
             'model_server': 'http://localhost:11434/v1',  #base_url，也称为 api_base
             'api_key':''}

tools = [
    {
        "mcpServers": {
            # enumeration of mcp server configs
            "filesystem": {
                "command": "npx",
                "args": [
                    "-y",
                    "@modelcontextprotocol/server-filesystem",
                    '.',
                ]
            },
            "sqlite" : {
                "command": "uvx",
                "args": [
                    "mcp-server-sqlite",
                    "--db-path",
                    "scores.db"
                ]
            }
        }
    }
]


tools = [
    {
        "name": "filesystem",  # 添加 name 字段
        "mcpServers": {
            "command": "npx",
            "args": [
                "-y",
                "@modelcontextprotocol/server-filesystem",
                '.',
            ]
        }
    },
    {
        "name": "sqlite",  # 添加 name 字段
        "mcpServers": {
            "command": "uvx",
            "args": [
                "mcp-server-sqlite",
                "--db-path",
                "scores.db"
            ]
        }
    }
]

agent = Assistant(
    llm=llm_cfg, 
    function_list=tools
)

KeyError: 'name'

# Step 2: Read the Spreadsheet

The agent will read the csv table by calling the file reading tool.

In [4]:
messages = [
    {
        'role': 'user',
        'content': 'Show me the contents in scores.csv.'
    }
]

In [5]:
response_plain_text = ''
for ret_messages in agent.run(messages):
    # `ret_messages` will contain all subsequent messages, consisting of interleaved assistant messages and tool responses
    response_plain_text = typewriter_print(ret_messages, response_plain_text)

[THINK]
Okay, the user wants to see the contents of scores.csv. Let me check which tool to use. The available functions include reading a file, so I should use filesystem-read_file. The parameters require the path to the file. Assuming scores.csv is in an allowed directory, I'll call that function with the path "scores.csv". I need to make sure the path is correct and accessible. If there's an issue, the function will return an error, but I'll proceed with the call as instructed.
[TOOL_CALL] filesystem-read_file
{"path": "scores.csv"}
[TOOL_RESPONSE] filesystem-read_file
id,score
100,85
101,90
102,88
103,95
104,80
105,92
106,87
107,100
108,83
109,98

[THINK]
Okay, the user asked for the contents of scores.csv, and I used the filesystem-read_file tool to get it. The response shows the CSV data with id and score columns. Now I need to present this information clearly. Let me check if there's any specific format the user wants. Since they just said "show me the contents," a simple display

In [6]:
messages += ret_messages # extending the context with new `ret_messages`.

# Step 3: Create a Database Table

Next, we create a database table and insert all the data that the agent just read.

In [7]:
messages.append({'role': 'user', 'content': "Create a table in the database, and fill it with the contents in the previous table."})

In [8]:
response_plain_text = ''

for ret_messages in agent.run(messages):
    response_plain_text = typewriter_print(ret_messages, response_plain_text)

[THINK]
Okay, the user wants me to create a table in the database and fill it with the data from the scores.csv file I just showed. Let me think about how to approach this.

First, I need to create a table. The data has two columns: id and score. Both are integers. So the SQL statement should define a table with those columns. Maybe name it 'scores' for simplicity. The SQL would be CREATE TABLE scores (id INTEGER, score INTEGER).

Next, I need to insert the data. The user probably expects me to use the data from the CSV. Since I have the CSV contents, I can generate INSERT statements for each row. But the user might not want to do that manually. Wait, the tools available include sqlite-create_table and sqlite-write_query. So first, I should call sqlite-create_table with the CREATE TABLE statement. Then, for each row, execute an INSERT. But inserting one by one might be inefficient. Alternatively, can I do a multi-row insert? Like INSERT INTO scores (id, score) VALUES (100,85), (101,90)

In [9]:
messages += ret_messages

# Step 4: Query the Highest Score

We query the highest score in the database. You can also perform other queries.

In [10]:
messages += [{'role': 'user', 'content': 'Query the highest score from the database.'}]

In [11]:
response_plain_text = ''

for ret_messages in agent.run(messages):
    response_plain_text = typewriter_print(ret_messages, response_plain_text)

[THINK]
Okay, the user wants to query the highest score from the database. Let me think about how to approach this.

First, I need to recall the structure of the database. The user created a table called 'scores' with columns 'id' and 'score'. The data has been populated with 10 entries. So, the table exists and has the necessary data.

The goal is to find the highest score. In SQL, the standard way to get the maximum value of a column is using the MAX() function. So the query should select the maximum value from the 'score' column in the 'scores' table.

The correct SQL query would be: SELECT MAX(score) FROM scores;

I need to use the sqlite-read_query function because this is a SELECT statement. The user might expect just the highest score as a result. Let me make sure there are no typos in the query. Also, since the table name and column names are correctly specified, there shouldn't be any syntax errors.

Once the query is executed, the result should return a single row with the ma

# Use the GUI

We have explored the capabilities of the Qwen-Agent framework and Qwen models for data manipulations with databases.

Even better, we can easily create a web UI for an agent!

In [12]:
from qwen_agent.gui import WebUI

agent = Assistant(
    name="Qwen Assistant",
    description="I'm a digital assistant powered by Qwen-Agent, ask me anything!",
    llm=llm_cfg,
    function_list=tools
)

WebUI(agent).run()
# Have fun!

2025-05-01 21:09:02,227 - mcp_manager.py - 110 - INFO - Initializing MCP tools from mcp servers: ['filesystem', 'sqlite']
2025-05-01 21:09:02,233 - mcp_manager.py - 245 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: filesystem
2025-05-01 21:09:03,425 - mcp_manager.py - 245 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: sqlite


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
